Vijay Kumar Jha Project 4th semester
# Finding best fit forecasing model for sales of different items in different shops of 1c software company by using different machine learning model.

## Dataset
Dataset contain total sales of every item in every shops per day from january 2013 to december 2015 having 29 lac row and 6 columns.
* date - date of recording(1034 date)
* date_block_num - code the month jan 2013 as 0 to dec 2015 as 33
* shop_id - 0 to 59 (total 59 shops)
* item_id - total 21806 unique item id
* item_price-
* itm_cnt_day-


In [1]:
# Basic Packages
import pandas as pd
import numpy as np
import random as rd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Time Series
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller,acf,pacf,arma_order_select_ic
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

In [3]:
# settings
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Import all of them
sales = pd.read_csv(r"C:\Users\VIJAY KR JHA\Desktop\python\competitive-data-science-predict-future-sales\sales_train.csv")

# settings
import warnings
warnings.filterwarnings("ignore")

item_cat = pd.read_csv(r"C:\Users\VIJAY KR JHA\Desktop\python\competitive-data-science-predict-future-sales\item_categories.csv")
item = pd.read_csv(r"C:\Users\VIJAY KR JHA\Desktop\python\competitive-data-science-predict-future-sales\items.csv")
sub = pd.read_csv(r"C:\Users\VIJAY KR JHA\Desktop\python\competitive-data-science-predict-future-sales\sample_submission.csv")
shops= pd.read_csv(r"C:\Users\VIJAY KR JHA\Desktop\python\competitive-data-science-predict-future-sales\shops.csv")
test = pd.read_csv(r"C:\Users\VIJAY KR JHA\Desktop\python\competitive-data-science-predict-future-sales\test.csv")

In [5]:
sales.shape

(2935849, 6)

In [6]:
sales.head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
5,10.01.2013,0,25,2564,349.00,1.0
6,02.01.2013,0,25,2565,549.00,1.0
7,04.01.2013,0,25,2572,239.00,1.0
8,11.01.2013,0,25,2572,299.00,1.0
9,03.01.2013,0,25,2573,299.00,3.0


In [7]:
sales.tail(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2935839,24.10.2015,33,25,7315,399.0,1.0
2935840,31.10.2015,33,25,7409,299.0,1.0
2935841,11.10.2015,33,25,7393,349.0,1.0
2935842,10.10.2015,33,25,7384,749.0,1.0
2935843,09.10.2015,33,25,7409,299.0,1.0
2935844,10.10.2015,33,25,7409,299.0,1.0
2935845,09.10.2015,33,25,7460,299.0,1.0
2935846,14.10.2015,33,25,7459,349.0,1.0
2935847,22.10.2015,33,25,7440,299.0,1.0
2935848,03.10.2015,33,25,7460,299.0,1.0


In [22]:
sales.describe(include = "all")

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2935849,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
unique,1034,NaN,NaN,NaN,NaN,NaN
top,28.12.2013,NaN,NaN,NaN,NaN,NaN
freq,9434,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,NaN,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,NaN,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,NaN,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,NaN,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,NaN,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00


In [23]:
sales["item_id"].unique()

array([22154,  2552,  2554, ...,  7610,  7635,  7640], dtype=int64)

In [24]:
sales["item_id"].unique().shape

(21807,)

In [25]:
sales["date"].unique().shape

(1034,)

In [26]:
sales["date_block_num"].unique().shape

(34,)

In [27]:
sales["shop_id"].unique().shape

(60,)

In [28]:
# formatting the date column correctly
sales.date = sales.date.apply(lambda x:datetime.datetime.strptime(x,"%d.%m.%Y"))
# check
print(sales.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
date              datetime64[ns]
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 134.4 MB
None


In [30]:
# Aggregate to monthly level the required metrics
monthly_sales = sales.groupby(["date_block_num","shop_id","item_id"])["date","item_price","item_cnt_day"].agg({"date":["min","max"],"item_price":"mean","item_cnt_day":"sum"})

## lets break down the line of code here:
# aggregete by date-block(month),shop_id and item_id
# select the columns date,item_price and item_cnt(sales)
# Provide a dictionary which says what aggregation to perform on which column
# min and max on the date
# average of the item_price
# sum of the sales

In [31]:
monthly_sales.head(20)

date            item_price item_cnt_day
                                      min        max       mean          sum
date_block_num shop_id item_id                                              
0              0       32      2013-01-03 2013-01-31      221.0          6.0
                       33      2013-01-03 2013-01-28      347.0          3.0
                       35      2013-01-31 2013-01-31      247.0          1.0
                       43      2013-01-31 2013-01-31      221.0          1.0
                       51      2013-01-13 2013-01-31      128.5          2.0
                       61      2013-01-10 2013-01-10      195.0          1.0
                       75      2013-01-17 2013-01-17       76.0          1.0
                       88      2013-01-16 2013-01-16       76.0          1.0
                       95      2013-01-06 2013-01-06      193.0          1.0
                       96      2013-01-10 2013-01-10       70.0          1.0
                       98      2013-01-04 2013-01-31      268.0         25.0
                       111     2013-01-17 2013-01-17       89.0          1.0
                       149     2013-01-11 2013-01-28       99.0          3.0
                       151     2013-01-16 2013-01-16       75.0          1.0
                       153     2013-01-09 2013-01-09      258.0          1.0
                       198     2013-01-10 2013-01-10      112.0          1.0
                       210     2013-01-05 2013-01-25      118.0          2.0
                       282     2013-01-04 2013-01-04      109.0          1.0
                       306     2013-01-22 2013-01-22       59.0          1.0
                       351     2013-01-21 2013-01-21       89.0          1.0

In [32]:
monthly_sales.shape

(1609124, 4)